In [12]:
!pip3 install -q python-dotenv langchain lanchain_core

ERROR: Could not find a version that satisfies the requirement lanchain_core (from versions: none)
ERROR: No matching distribution found for lanchain_core

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


In [54]:
model = ChatOpenAI(temperature=0,streaming=True)
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}. 200 Words")
parser = StrOutputParser()
chain = prompt | model | parser


In [52]:
for chain in chain.stream("parrot"):
    print(chain,end="",flush=True)

Why did the parrot wear a raincoat?

Because he wanted to be polyunsaturated!

Why did the parrot sit on the clock?

Because he wanted to be on "time"!

Why did the parrot go to the doctor?

Because he was feeling a little "tweet"!

Why did the parrot join the circus?

Because he wanted to be a "bird" performer!

Why did the parrot bring a ladder to the bar?

Because he heard the drinks were on the house!

Why did the parrot refuse to eat his vegetables?

Because he was already stuffed with crackers!

Why did the parrot go to school?

Because he wanted to be a "smarty beak"!

Why did the parrot start a band?

Because he had a great set of "tweet" harmonies!

Why did the parrot go to the beach?

Because he wanted to soak up some "sun"flower seeds!

Why did the parrot become a detective?

Because he had a knack for "cracking" cases!

In [55]:
async for chunk in chain.astream("parrot"):
    print(chunk, end="", flush=True)

Why did the parrot wear a raincoat?

Because he wanted to be polyunsaturated!

Why did the parrot sit on the clock?

Because he wanted to be on "time"!

Why did the parrot go to the doctor?

Because he was feeling a little "tweet"!

Why did the parrot join the circus?

Because he wanted to be a "bird" performer!

Why did the parrot bring a ladder to the bar?

Because he heard the drinks were on the house!

Why did the parrot refuse to eat his vegetables?

Because he was on a "seed" only diet!

Why did the parrot go to school?

Because he wanted to be a "smarty beak"!

Why did the parrot become a detective?

Because he had a knack for "cracking" cases!

Why did the parrot start a band?

Because he had a talent for "tweeting" music!

Why did the parrot go to the beach?

Because he wanted to "seas" the day!

In [56]:
async for event in model.astream_events("parrot", version="v1"):
    print(event, flush=True)


{'event': 'on_chat_model_start', 'run_id': 'a292c464-3ed8-47bf-a634-6269554dc9c8', 'name': 'ChatOpenAI', 'tags': [], 'metadata': {}, 'data': {'input': 'parrot'}, 'parent_ids': []}


/home/debian/.local/lib/python3.11/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


{'event': 'on_chat_model_stream', 'run_id': 'a292c464-3ed8-47bf-a634-6269554dc9c8', 'tags': [], 'metadata': {}, 'name': 'ChatOpenAI', 'data': {'chunk': AIMessageChunk(content='', id='run-a292c464-3ed8-47bf-a634-6269554dc9c8')}, 'parent_ids': []}
{'event': 'on_chat_model_stream', 'run_id': 'a292c464-3ed8-47bf-a634-6269554dc9c8', 'tags': [], 'metadata': {}, 'name': 'ChatOpenAI', 'data': {'chunk': AIMessageChunk(content='A', id='run-a292c464-3ed8-47bf-a634-6269554dc9c8')}, 'parent_ids': []}
{'event': 'on_chat_model_stream', 'run_id': 'a292c464-3ed8-47bf-a634-6269554dc9c8', 'tags': [], 'metadata': {}, 'name': 'ChatOpenAI', 'data': {'chunk': AIMessageChunk(content=' par', id='run-a292c464-3ed8-47bf-a634-6269554dc9c8')}, 'parent_ids': []}
{'event': 'on_chat_model_stream', 'run_id': 'a292c464-3ed8-47bf-a634-6269554dc9c8', 'tags': [], 'metadata': {}, 'name': 'ChatOpenAI', 'data': {'chunk': AIMessageChunk(content='rot', id='run-a292c464-3ed8-47bf-a634-6269554dc9c8')}, 'parent_ids': []}
{'event'

In [57]:
events = []
async for event in model.astream_events("parrot", version="v1"):
    events.append(event)
event_types = {event["event"] for event in events}
print("Unique event types:", event_types)


Unique event types: {'on_chat_model_stream', 'on_chat_model_end', 'on_chat_model_start'}


In [62]:
async for event in model.astream_events("parrot", version="v1"):
    if event["event"] == "on_chat_model_start":
        print("Stream started...", flush=True)
    elif event["event"] == "on_chat_model_stream":
        print(event["data"]["chunk"].content, end="", flush=True)

Stream started...
A parrot is a type of bird known for its colorful feathers, ability to mimic sounds, and intelligence. They are often kept as pets and can be trained to talk and perform tricks. Parrots are found in tropical and subtropical regions around the world.

In [4]:
from typing_extensions import TypedDict
from typing import Annotated
from langgraph.graph.message import add_messages

# Add messages essentially does this with more
# robust handling
# def add_messages(left: list, right: list):
#     return left + right


class State(TypedDict):
    messages: Annotated[list, add_messages]


In [5]:
from langchain_core.tools import tool


@tool
def search(query: str):
    """Call to surf the web."""
    # This is a placeholder, but don't tell the LLM that...
    return ["Cloudy with a chance of hail."]


tools = [search]


In [6]:
from langgraph.prebuilt import ToolNode

tool_node = ToolNode(tools)


In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")


In [8]:
model = model.bind_tools(tools)


In [9]:
from langgraph.graph import StateGraph, END, START
from langchain_core.runnables import RunnableConfig
from typing import Literal


# Define the function that determines whether to continue or not
def should_continue(state: State) -> Literal["__end__", "tools"]:
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return END
    # Otherwise if there is, we continue
    else:
        return "tools"


# Define the function that calls the model
async def call_model(state: State, config: RunnableConfig):
    messages = state["messages"]
    # Note: Passing the config through explicitly is required for python < 3.11
    # Since context var support wasn't added before then: https://docs.python.org/3/library/asyncio-task.html#creating-tasks
    response = await model.ainvoke(messages, config)
    # We return a list, because this will get added to the existing list
    return {"messages": response}


In [10]:
workflow = StateGraph(State)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

workflow.add_edge("tools", "agent")

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()


In [13]:
from langchain_core.messages import HumanMessage

inputs = [HumanMessage(content="what is the weather in sf. write 200 words")]
async for event in app.astream_events({"messages": inputs}, version="v1"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI or Anthropic usually means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end=" ")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")


--
Starting tool: search with inputs: {'query': 'Weather in San Francisco'}
Done tool: search
Tool output was: ['Cloudy with a chance of hail.']
--
The  current  weather  in  San  Francisco  is  cloudy  with  a  chance  of  hail .  The  weather  conditions  in  the  city  are  expected  to  be  unsett led  with  the  possibility  of  hail  showers .  It  is  recommended  to  stay  updated  on  the  weather  forecast  and  be  prepared  for  sudden  changes  in  weather  patterns .  San  Francisco ,  known  for  its  micro cl imates ,  can  experience  varying  weather  conditions  across  different  neighborhoods .  It  is  advisable  to  carry  an  umbrella  or  rain coat  when  heading  out  to  ensure  you  are  prepared  for  any  unexpected  rain  or  hail  showers .  Additionally ,  it  is  important  to  drive  carefully  and  safely  on  the  roads  during  adverse  weather  conditions  to  ensure  your  safety  and  the  safety  of  others .  Stay  informed  about  the  weathe